In [254]:
import pandas as pd
import numpy as np

In [255]:
df = pd.read_csv("./data/Titanic/train.csv")

df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [256]:
# Pclass, Sex : 범주형
# 나머지 : 연속형

In [257]:
# 데이터 전처리
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]

display(df.head())
print(df.dtypes)

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,3,male,22.0,1,0,7.2500,0
1,1,female,38.0,1,0,71.2833,1
2,3,female,26.0,0,0,7.9250,1
3,1,female,35.0,1,0,53.1000,1
4,3,male,35.0,0,0,8.0500,0


Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Survived      int64
dtype: object


In [258]:
df['Pclass'] = df['Pclass'].astype('object')
display(df.head())
print(df.dtypes)

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,3,male,22.0,1,0,7.2500,0
1,1,female,38.0,1,0,71.2833,1
2,3,female,26.0,0,0,7.9250,1
3,1,female,35.0,1,0,53.1000,1
4,3,male,35.0,0,0,8.0500,0


Pclass       object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Survived      int64
dtype: object


In [259]:
target = 'Survived'
y = df[target].values # target 값 분리
df = df.drop(columns=[target])

display(y[:5])
display(df.head(3))

array([0, 1, 1, 1, 0])

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833
2,3,female,26.0,0,0,7.9250


In [260]:
cat_indicator = np.array([True if str(dtype) == 'object' else False for dtype in df.dtypes]) # 범주형 / 연속형 여부
display(cat_indicator)

array([ True,  True, False, False, False, False])

In [261]:
cat_idxs = list(np.where(cat_indicator == True)[0]) # 범주형 열 인덱스
con_idxs = list(np.where(cat_indicator != True)[0]) # 연속형 열 인덱스

cat_cols = df.columns[cat_idxs].tolist() # 범주형 열 레이블
con_cols = df.columns[con_idxs].tolist() # 연속형 열 레이블

print(cat_idxs, con_idxs, sep=', ')
print(cat_cols, con_cols, sep=', ')

[0, 1], [2, 3, 4, 5]
['Pclass', 'Sex'], ['Age', 'SibSp', 'Parch', 'Fare']


In [262]:
# train / valid / test 분리

df["Set"] = np.random.choice(["train", "valid", "test"], p=[.65, .15, .2], size=df.shape[0])
train_indices = df.loc[df["Set"] == "train"].index
valid_indices = df.loc[df["Set"] == "valid"].index
test_indices = df.loc[df["Set"] == "test"].index


df.drop(columns=["Set"], inplace=True)

print(test_indices[:3], valid_indices[:3], test_indices[:3], sep=', ')
display(df.head())


Index([1, 8, 13], dtype='int64'), Index([5, 7, 11], dtype='int64'), Index([1, 8, 13], dtype='int64')


,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833
2,3,female,26.0,0,0,7.9250
3,1,female,35.0,1,0,53.1000
4,3,male,35.0,0,0,8.0500


In [263]:
temp = df.copy().fillna("MissingValue")
nan_mask = temp.ne("MissingValue").astype(int)

display(temp.tail(5))
display(nan_mask.tail(5))

/tmp/ipykernel_511355/3606198463.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  temp = df.copy().fillna("MissingValue")


,Pclass,Sex,Age,SibSp,Parch,Fare
886,2,male,27.0,0,0,13.00
887,1,female,19.0,0,0,30.00
888,3,female,MissingValue,1,2,23.45
889,1,male,26.0,0,0,30.00
890,3,male,32.0,0,0,7.75


,Pclass,Sex,Age,SibSp,Parch,Fare
886,1,1,1,1,1,1
887,1,1,1,1,1,1
888,1,1,0,1,1,1
889,1,1,1,1,1,1
890,1,1,1,1,1,1


In [264]:
nan_mask.values[test_indices]

array([[1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       ...,
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1]])

In [271]:
y_d = y[test_indices].reshape(-1, 1)

print(y_d[:3])

[[1]
 [1]
 [0]]


In [266]:
y_d.shape

(170, 1)

In [267]:
from sklearn.preprocessing import LabelEncoder

# 범주형 데이터 : 라벨 인코딩 / 연속형 데이터 : 결측치 처리
cat_dims = [] # 범주형 데이터 class 개수
# 범주형 데이터 처리
for col in cat_cols:
    df[col] = df[col].fillna("MissingValue")
    l_enc = LabelEncoder()
    df[col] = l_enc.fit_transform(df[col].values)
    cat_dims.append(len(l_enc.classes_))

print(cat_dims)

[3, 2]


/tmp/ipykernel_511355/3345624523.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna("MissingValue")


In [272]:
def data_split(X,y,nan_mask,indices):
    x_d = {
        'data': X.values[indices],
        'mask': nan_mask.values[indices]
    }
    
    if x_d['data'].shape != x_d['mask'].shape:
        raise'Shape of data not same as that of nan mask! '
        
    y_d = {
        'data': y[indices].reshape(-1, 1)
    } 
    return x_d, y_d

In [273]:
# 연속형 데이터 처리
for col in con_cols:
    df.fillna(df.loc[train_indices, col].mean(), inplace=True)

# Target 데이터 처리
task = 'classification'
if task != 'regression':
    l_enc = LabelEncoder()
    y = l_enc.fit_transform(y)

In [276]:
X_train, y_train = data_split(df, y, nan_mask, train_indices)
X_valid, y_valid = data_split(df,y,nan_mask,valid_indices)
X_test, y_test = data_split(df,y,nan_mask,test_indices)

In [279]:
display(X_train)
display(y_train)

{'data': array([[ 2.   ,  1.   , 22.   ,  1.   ,  0.   ,  7.25 ],
        [ 2.   ,  0.   , 26.   ,  0.   ,  0.   ,  7.925],
        [ 0.   ,  0.   , 35.   ,  1.   ,  0.   , 53.1  ],
        ...,
        [ 1.   ,  1.   , 27.   ,  0.   ,  0.   , 13.   ],
        [ 0.   ,  1.   , 26.   ,  0.   ,  0.   , 30.   ],
        [ 2.   ,  1.   , 32.   ,  0.   ,  0.   ,  7.75 ]]),
 'mask': array([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        ...,
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}

{'data': array([[0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
    

In [284]:
# 각각의 연속형 features에 대한 평균 / 표준편차 구하기
train_mean = np.array(X_train['data'][:, con_idxs], dtype=np.float32).mean(axis=0)
train_std = np.array(X_train['data'][:, con_idxs], dtype=np.float32).std(axis=0)

train_std = np.where(train_std < 1e-6, 1e-6, train_std) # 표준편차가 너무 작아지는 것을 방지
print(train_mean, train_std)

[29.89076     0.5443478   0.38782609 31.338875  ] [13.5233965  1.1606737  0.798384  48.15419  ]
